In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

In [2]:
cassandra_host = "192.168.31.188"
cassandra_user = "cassandra"
cassandra_pwd  = "cassandra"
cassandra_port = 9042
key_space      = "LogAnalysis"
table_name     = "ApacheLogs"
kafka_server   = "192.168.31.188:9092"
kafka_topic    = "logs"

In [3]:
#Spark Session creation configured to interact with MongoDB
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector_2.12:3.0.0,com.datastax.spark:spark-cassandra-connector-driver_2.12:3.0.0").\
config("spark.cassandra.connection.host",cassandra_host).\
config("spark.cassandra.auth.username",cassandra_user).\
config("spark.cassandra.auth.password",cassandra_pwd).\
getOrCreate()

:: loading settings :: url = jar:file:/Users/ngohongthai/miniconda3/envs/data-engineering/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ngohongthai/.ivy2/cache
The jars for the packages stored in: /Users/ngohongthai/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector-driver_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5797d75f-295a-4c1d-bf0b-0d855e7123e2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found com.datastax.spark#spa

In [11]:
#Read data from Kafka topic
#earliest
split_logic = split(col("url"),"\.").getItem(1)
log_data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers",kafka_server)\
  .option("subscribe", kafka_topic)\
  .option("startingOffsets", "earliest")\
  .load()\
  .selectExpr("split(value,',')[1] as host",
              "split(value,',')[2] as time",
              "split(value,',')[3] as method",
              "split(value,',')[4] as url",
              "split(value,',')[5] as response",
              "split(value,',')[6] as bytes"
             )\
  .withColumn("time_added",unix_timestamp())\
  .withColumn("extension",when(split_logic.isNull(),"None").otherwise(split_logic))

In [12]:
log_data.printSchema()

root
 |-- host: string (nullable = true)
 |-- time: string (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response: string (nullable = true)
 |-- bytes: string (nullable = true)
 |-- time_added: long (nullable = true)
 |-- extension: string (nullable = true)



In [13]:
# print out the log file
log_data.writeStream.format("console").outputMode("append").start().awaitTermination()

23/08/16 00:19:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/66/6rbnzyf91v9d5v8nb46hmn0h0000gn/T/temporary-625cd4bc-e201-448f-8bf6-5f29aea48d6b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/08/16 00:19:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+----------+---------+
|                host|         time|            method|                 url|            response|               bytes|time_added|extension|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+----------+---------+
|"@metadata":{"bea...|"type":"_doc"|"version":"8.9.0"}|"message":"187.35...| like Gecko) Vers...|"input":{"type":"...|1692119996|       35|
|"@metadata":{"bea...|"type":"_doc"|"version":"8.9.0"}|"message":"181.17...| like Gecko) Chro...|"input":{"type":"...|1692119996|      179|
|"@metadata":{"bea...|"type":"_doc"|"version":"8.9.0"}|"host":{"name":"d...| "log":{"offset":609|"file":{"path":"/...|1692119996|     None|
|"@metadata":{"bea...|"type":"_doc"|"version":"8.9.0"}|"ecs":{"

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/ngohongthai/miniconda3/envs/data-engineering/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/ngohongthai/miniconda3/envs/data-engineering/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/ngohongthai/miniconda3/envs/data-engineering/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|[7B 22 40 74 69 6...| logs|        0|   636|2023-08-16 00:23:...|            0|
+----+--------------------+-----+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+----------+---------+
|                host|         time|            method|                 url|            response|               bytes|time_added|extension|
+--------------------+-------------+------------------+--------------------+-------------------